In [6]:
import pdfplumber
import pandas as pd

ruta_pdf = "../data/350_Revista CIFRAS Digital Mayo 2025 - Costos x rubro.xls - CostosUnitarios.pdf"

lineas_extraidas = []

with pdfplumber.open(ruta_pdf) as pdf:
    for i, pagina in enumerate(pdf.pages):
        texto = pagina.extract_text()
        if texto:
            print(f"Página {i+1} ✅ OK")
            for linea in texto.split("\n"):
                lineas_extraidas.append({"pagina": i+1, "texto": linea.strip()})
        else:
            print(f"Página {i+1} ❌ VACÍA")

# Crear DataFrame
df_lineas = pd.DataFrame(lineas_extraidas)




CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Página 1 ✅ OK
Página 2 ✅ OK
Página 3 ✅ OK
Página 4 ✅ OK
Página 5 ✅ OK
Página 6 ✅ OK
Página 7 ✅ OK
Página 8 ✅ OK


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Página 9 ✅ OK


In [7]:
df_numericas = df_lineas[df_lineas["texto"].str.contains(r"^\d", regex=True)]

df_numericas

,pagina,texto
11,1,00 HONORARIOS PROFESIONALES
16,1,00.01 Tareas de Proyecto / Dirección o Conducc...
17,1,01 PRELIMINARES DE OBRA
19,1,"01.01 Cartel de Obra m2 $ 45.523,88 $ 42.645,3..."
20,1,"01.02 Cerco de Obra m $ 23.616,17 $ 15.671,02 ..."
...,...,...
323,9,20 VARIOS
324,9,"20.01 Ayuda de gremio m2 $ 0,00 $ 5.186,11 $ 5..."
325,9,"20.02 Conductos ventilación, incl. sombrerete ..."
326,9,20.03 Limpieza periódica y final de obra de ob...


In [8]:
import re
import pandas as pd

# Partimos de df_numericas (con las líneas que comienzan con número)
materiales = []
rubro_actual = None

for _, fila in df_numericas.iterrows():
    texto = fila["texto"].strip()

    # Detectar rubros: líneas como "03 ESTRUCTURAS"
    if re.match(r"^\d{2}\s+.+", texto):
        rubro_actual = texto
        continue

    # Detectar líneas de materiales: "03.01 Descripción unidad $ x $ y $ z"
    match = re.match(
        r"^(\d{2}\.\d{2})\s+(.*?)\s+([^\s]+)\s+\$?\s*([\d.,]+)\s+\$?\s*([\d.,]+)\s+\$?\s*([\d.,]+)",
        texto
    )
    if match:
        codigo, descripcion, unidad, mat, ejec, total = match.groups()
        try:
            materiales.append({
                "codigo": codigo,
                "descripcion": descripcion.strip(),
                "unidad": unidad.strip(),
                "costo_material": float(mat.replace(".", "").replace(",", ".")),
                "costo_ejecucion": float(ejec.replace(".", "").replace(",", ".")),
                "costo_total": float(total.replace(".", "").replace(",", ".")),
                "rubro": rubro_actual
            })
        except:
            continue

df_materiales = pd.DataFrame(materiales)

In [9]:
df_materiales

,codigo,descripcion,unidad,costo_material,costo_ejecucion,costo_total,rubro
0,01.01,Cartel de Obra,m2,45523.88,42645.32,88169.20,01 PRELIMINARES DE OBRA
1,01.02,Cerco de Obra,m,23616.17,15671.02,39287.19,01 PRELIMINARES DE OBRA
2,01.03,Limpieza inicial terreno; retiros grales.,m2,379.70,3165.42,3545.12,01 PRELIMINARES DE OBRA
3,01.04,Nivelación y replanteo de obra,m2,113.91,6646.79,6760.70,01 PRELIMINARES DE OBRA
4,01.05,"Obrador, construcciones provisorias",m2,37419.54,111193.39,148612.92,01 PRELIMINARES DE OBRA
...,...,...,...,...,...,...,...
231,19.06,Matafuegos ABC 5kg,u,129552.51,15581.53,145134.04,19 EQUIPAMIENTO
232,20.01,Ayuda de gremio,m2,0.00,5186.11,5186.11,20 VARIOS
233,20.02,"Conductos ventilación, incl. sombrerete y reja",m,25380.44,93258.87,118639.31,20 VARIOS
234,20.03,Limpieza periódica y final de obra de obra,m2,2475.55,10131.70,12607.26,20 VARIOS


In [12]:
import sys
import os

# Suponiendo que el notebook está en /CRRLN/notebooks/
sys.path.append(os.path.abspath(".."))

from backend.db import engine
from backend.models.material import Material
from backend.models.rubro import Rubro
from sqlmodel import Session, select
from datetime import date
import re

# Asegurate de que df_materiales exista y tenga columna "rubro"
rubro_cache = {}

with Session(engine) as session:
    for _, fila in df_materiales.iterrows():
        rubro_texto = fila["rubro"].strip()

        # Extraer código y nombre de rubro
        match = re.match(r"^(\d{2})\s+(.+)", rubro_texto)
        if not match:
            print(f"⚠️ Rubro mal formateado: {rubro_texto}")
            continue

        codigo_rubro, nombre_rubro = match.groups()

        # Buscar o crear rubro
        if rubro_texto not in rubro_cache:
            rubro = session.exec(select(Rubro).where(Rubro.codigo == codigo_rubro)).first()
            if not rubro:
                rubro = Rubro(nombre=nombre_rubro, codigo=codigo_rubro)
                session.add(rubro)
                session.commit()
                session.refresh(rubro)
            rubro_cache[rubro_texto] = rubro.id

        # Crear objeto Material
        material = Material(
            codigo=fila["codigo"],
            descripcion=fila["descripcion"],
            unidad=fila["unidad"],
            costo_material=fila["costo_material"],
            costo_ejecucion=fila["costo_ejecucion"],
            costo_total=fila["costo_total"],
            rubro_id=rubro_cache[rubro_texto],
            fuente="Revista Cifras - Mayo 2025",
            fecha=date(2025, 5, 1)
        )
        session.add(material)

    session.commit()
    print("✅ Todos los materiales fueron insertados.")



2025-05-12 23:47:36,017 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-12 23:47:36,018 INFO sqlalchemy.engine.Engine SELECT rubro.id, rubro.nombre, rubro.codigo 
FROM rubro 
WHERE rubro.codigo = ?
2025-05-12 23:47:36,019 INFO sqlalchemy.engine.Engine [cached since 125s ago] ('01',)
2025-05-12 23:47:36,025 INFO sqlalchemy.engine.Engine INSERT INTO rubro (nombre, codigo) VALUES (?, ?)
2025-05-12 23:47:36,027 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('PRELIMINARES DE OBRA', '01')
2025-05-12 23:47:36,041 INFO sqlalchemy.engine.Engine COMMIT
2025-05-12 23:47:36,054 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-12 23:47:36,059 INFO sqlalchemy.engine.Engine SELECT rubro.id, rubro.nombre, rubro.codigo 
FROM rubro 
WHERE rubro.id = ?
2025-05-12 23:47:36,061 INFO sqlalchemy.engine.Engine [generated in 0.00215s] (1,)
2025-05-12 23:47:36,077 INFO sqlalchemy.engine.Engine INSERT INTO material (codigo, descripcion, unidad, costo_material, costo_ejecucion, costo_total, r

In [11]:
from backend.db import engine
from backend.models.material import Material
from backend.models.rubro import Rubro
from sqlmodel import SQLModel

SQLModel.metadata.create_all(engine)
print("✅ Tablas creadas correctamente.")

2025-05-12 23:47:30,773 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-12 23:47:30,775 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("material")
2025-05-12 23:47:30,776 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-12 23:47:30,779 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("material")
2025-05-12 23:47:30,779 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-12 23:47:30,782 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("rubro")
2025-05-12 23:47:30,782 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-12 23:47:30,785 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("rubro")
2025-05-12 23:47:30,785 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-12 23:47:30,787 INFO sqlalchemy.engine.Engine 
CREATE TABLE rubro (
	id INTEGER NOT NULL, 
	nombre VARCHAR NOT NULL, 
	codigo VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2025-05-12 23:47:30,788 INFO sqlalchemy.engine.Engine [no key 0.00084s] ()
2025-05-12 23:47:30,816 INFO sqlalchemy.engine.Engine 
CREATE